In [3]:
import numpy as np

class BellmanEquationSolver:
    """使用矩阵向量方式求解贝尔曼方程"""
    
    def __init__(self, P, R, gamma=0.9):
        """
        初始化求解器
        
        参数:
        P: 状态转移概率矩阵 (n_states, n_states)
           P[i,j] 表示从状态i转移到状态j的概率
        R: 奖励向量 (n_states,)
           R[i] 表示状态i的即时奖励
        gamma: 折扣因子 (0 <= gamma < 1)
        """
        self.P = np.array(P)
        self.R = np.array(R)
        self.gamma = gamma
        self.n_states = len(R)
        
    def solve_closed_form(self):
        """
        使用闭式解求解贝尔曼方程
        
        贝尔曼方程: V = R + γPV
        变换: V - γPV = R
              (I - γP)V = R
              V = (I - γP)^(-1)R
        
        返回: 状态值函数 V
        """
        I = np.eye(self.n_states)
        A = I - self.gamma * self.P
        V = np.linalg.solve(A, self.R)
        return V
    
    def solve_iterative(self, max_iter=1000, tol=1e-6):
        """
        使用迭代法求解贝尔曼方程
        
        迭代公式: V_{k+1} = R + γPV_k
        
        参数:
        max_iter: 最大迭代次数
        tol: 收敛阈值
        
        返回: (状态值函数 V, 迭代次数)
        """
        V = np.zeros(self.n_states)
        
        for i in range(max_iter):
            V_new = self.R + self.gamma * self.P @ V
            
            # 检查收敛
            if np.max(np.abs(V_new - V)) < tol:
                print(f"迭代法在第 {i+1} 次迭代后收敛")
                return V_new, i+1
            
            V = V_new
        
        print(f"达到最大迭代次数 {max_iter}")
        return V, max_iter
    
    def solve_gauss_seidel(self, max_iter=1000, tol=1e-6):
        """
        使用 Gauss-Seidel 迭代法求解
        
        参数:
        max_iter: 最大迭代次数
        tol: 收敛阈值
        
        返回: (状态值函数 V, 迭代次数)
        """
        V = np.zeros(self.n_states)
        
        for iteration in range(max_iter):
            V_old = V.copy()
            
            for i in range(self.n_states):
                V[i] = self.R[i] + self.gamma * np.dot(self.P[i, :], V)
            
            # 检查收敛
            if np.max(np.abs(V - V_old)) < tol:
                print(f"Gauss-Seidel法在第 {iteration+1} 次迭代后收敛")
                return V, iteration+1
            
        print(f"达到最大迭代次数 {max_iter}")
        return V, max_iter



In [ ]:
from GridWorld import GridWorld, print_grid
from Bellman import BellmanEquationSolver


obstacles = {6, 7, 12, 16, 18, 21}
goal_states = {17}

action_map = {
    0: 0, 1: 0, 2: 0, 3: 1, 4: 1,
    5: 3, 6: 3, 7: 0, 8: 1, 9: 1,  # 障碍6,7也有策略
    10: 3, 11: 2, 12: 1, 13: 0, 14: 1,  # 障碍12也有策略
    15: 3, 16: 0, 18: 2, 19: 1,  # 障碍16,18也有策略
    20: 3, 21: 0, 22: 3, 23: 2, 24: 2  # 障碍21也有策略
}

# 创建一个5x5的网格世界
gw = GridWorld(size=5, n_states=25, action_map=action_map, gamma=0.9,
               obstacles=obstacles, goal_states=goal_states)
P, R = gw.build()

be = BellmanEquationSolver(P, R, gamma=0.9)
V_iter, iterations = be.solve_iterative()
V_close = be.solve_closed_form()
print(f"迭代法求得： V (after {iterations} iterations)")
print_grid(gamma=0.9, V=V_iter, size=5, obstacles=obstacles, goal_states=goal_states)

print(f"解析式求解： V")
print_grid(gamma=0.9, V=V_close, size=5, obstacles=obstacles, goal_states=goal_states)

print("\n" + "="*60)
print("图例说明：")
print("  [value]G  → 目标状态 (Goal)")
print("  (value)X  → 障碍区域 (Obstacle)")
print("   value    → 正常可达区域")
print("="*60)

迭代法在第 133 次迭代后收敛，误差: 9.12e-07
iterative V (after 133 iterations)

状态值函数 (γ = 0.9):

       col0      col1      col2      col3      col4   
row0    3.49      3.87      4.30      4.78      5.31  
row1    3.14   (  3.49)X (  4.78)X    5.31      5.90  
row2    2.82      2.54   ( 10.00)X    5.90      6.56  
row3    2.54   ( 10.00)X [ 10.00]G ( 10.00)X    7.29  
row4    2.29   (  9.00)X   10.00      9.00      8.10  
closed-form V

状态值函数 (γ = 0.9):

       col0      col1      col2      col3      col4   
row0    3.49      3.87      4.30      4.78      5.31  
row1    3.14   (  3.49)X (  4.78)X    5.31      5.90  
row2    2.82      2.54   ( 10.00)X    5.90      6.56  
row3    2.54   ( 10.00)X [ 10.00]G ( 10.00)X    7.29  
row4    2.29   (  9.00)X   10.00      9.00      8.10  

图例说明：
  [value]G  → 目标状态 (Goal)
  (value)X  → 障碍区域 (Obstacle)
   value    → 正常可达区域


In [ ]:

obstacles = {6, 7, 12, 16, 18, 21}
goal_states = {17}

action_map = {
    0: 0, 1: 0, 2: 0, 3: 1, 4: 1,
    5: 3, 6: 3, 7: 0, 8: 1, 9: 1,  # 障碍6,7也有策略
    10: 3, 11: 2, 12: 1, 13: 0, 14: 1,  # 障碍12也有策略
    15: 3, 16: 0, 18: 2, 19: 1,  # 障碍16,18也有策略
    20: 3, 21: 0, 22: 3, 23: 2, 24: 2  # 障碍21也有策略
}

# 创建一个5x5的网格世界
gw = GridWorld(size=5, n_states=25, action_map=action_map, gamma=0.9,
               obstacles=obstacles, goal_states=goal_states)
P, R = gw.build()

be = BellmanEquationSolver(P, R, gamma=0.9)
V_iter, iterations = be.solve_iterative()
V_close = be.solve_closed_form()
print(f"迭代法求得： V (after {iterations} iterations)")
print_grid(gamma=0.9, V=V_iter, size=5, obstacles=obstacles, goal_states=goal_states)

print(f"解析式求解： V")
print_grid(gamma=0.9, V=V_close, size=5, obstacles=obstacles, goal_states=goal_states)

print("\n" + "="*60)
print("图例说明：")
print("  [value]G  → 目标状态 (Goal)")
print("  (value)X  → 障碍区域 (Obstacle)")
print("   value    → 正常可达区域")
print("="*60)